## Branch and Bound Method for Integer Programming

First we import these modules

In [1]:
import scipy
import numpy as np
import itertools

#### Branch and Bound

In [55]:
def branch_and_bound(c, A, b, lb, ub, isMax = False, depth = 1):
    """
    Implementation of branch and bound method
    
    Args:
        c: profit vector
        isMax: whether it is a maximization or minimization problem
        lb, ub: vectors indicate the lower bound and upper bound of all decision variables
        depth: the depth of the tree
    """
    n = len(c)

    # Optimal solution for LP relaxation
    c = -c if isMax else c
    res = scipy.optimize.linprog(c=c, A_ub=A, b_ub=b, bounds = list(zip(lb, ub)))

    # Check if LP is feasible
    if not res.success:
        return [], -np.inf if isMax else np.inf, depth
    
    # Candidate for the optimal value and the objective value
    x_candidate, z_candidate = res.x, res.fun
    z_candidate = -z_candidate if isMax else z_candidate
    
    print(x_candidate)
    print(z_candidate)
    
    # Correct sign for maximization
    is_int = True 
    for i in range(n):
        # If x_i is not integer
        if not x_candidate[i] % 1 == 0:
            is_int = False
            break

    if is_int:
        return x_candidate, z_candidate, depth
    
    # Branch on the first non-integer variable
    for i in range(n):
        if not x_candidate[i] % 1 == 0:
            # Left branch: x_i <= floor(x_i)
            left_lb = np.copy(lb)
            left_ub = np.copy(ub)
            left_ub[i] = np.floor(x_candidate[i])
            x_left, z_left, depth_left = branch_and_bound(c, A, b, left_lb, left_ub, isMax, depth + 1)

            # Right branch: x_i >= ceil(x_i)
            right_lb = np.copy(lb)     
            right_ub = np.copy(ub) 
            right_lb[i] = np.ceil(x_candidate[i])
            x_right, z_right, depth_right = branch_and_bound(c, A, b, right_lb, right_ub, isMax, depth + 1)

            # Return the best solution
            # For max problem, return the branch with greater z
            if isMax:
                if(abs(z_left) > abs(z_right)):
                    return x_left, abs(z_left), depth_left
                else: 
                    return x_right, abs(z_right), depth_right
            # For min problem, return the branch with smaller z
            else:
                if(abs(z_left) < abs(z_right)):
                    return x_left, abs(z_left), depth_left
                else: 
                    return x_right, abs(z_right), depth_right  

#### Constraint vector A

In [30]:
def matrix_A(num_vars, ov_pairs):
    # Number of columns is the number of variables
    cols = num_vars
    # Number of rows is the number of all possible pair of the variables
    rows = len(ov_pairs)
    A = np.zeros((rows, cols))

    # All possible permutations of column entry
    # permutations = list(itertools.combinations(range(cols), 2))

    # Generate matrix A by letting the two column entries in each permutation (row) equal to 1 
    for i in range(rows):
        A[i][ov_pairs[i][0]] = 1
        A[i][ov_pairs[i][1]] = 1
    return A

#### Test

In [22]:
def is_overlapping(s1, e1, s2, e2):
    if (e1 <= s2 or s1 >= e2):
        return False 
    return True

def overlapping_pairs(start, end):
    d = {}
    n = len(start)
    
    for i in range(n):
        s1, e1 = start[i], end[i]
        d[i] = []
        for j in range(i+1, n):
            s2, e2 = start[j], end[j]
            if (is_overlapping(s1, e1, s2, e2)):
                d[i] += [j]
    pairs = []
    for k in d:
        for val in d[k]:
            pairs.append([k, val])
    
    return pairs
    

In [58]:
# Objective Function Coefficient 
c = np.array([50,10,40,70])

# Start, end times
start = [1,2,3,3]
end = [3,4,5,6]

# Constraint
cts = overlapping_pairs(start, end)
decision_vars_count = len(start)
A = matrix_A(decision_vars_count, cts)
b = np.ones(len(cts))
print(A)
print(b)

# Lower bound for each decision var 
lb = np.zeros(decision_vars_count)
# Upper bound for each decision var 
ub = np.ones(decision_vars_count)

x_optimal, f_optimal, depth_optimal = branch_and_bound(c, A, b, lb, ub, True)
print("========= Optimal Solutions =========")

print("Variable:")
for idx, var in enumerate(x_optimal): 
    print(f">  x{idx} = {var}")

print("Objective Function:")
print(f">  f = {f_optimal}")
print(f"Tree depth: {depth_optimal}")

[[1. 1. 0. 0.]
 [0. 1. 1. 0.]
 [0. 1. 0. 1.]
 [0. 0. 1. 1.]]
[1. 1. 1. 1.]
[ 1. -0.  0.  1.]
120.0
========= Optimal Solutions =========
Variable:
>  x0 = 1.0
>  x1 = -0.0
>  x2 = 0.0
>  x3 = 1.0
Objective Function:
>  f = 120.0
Tree depth: 1
